In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [6]:
fake = pd.read_csv('Fake.csv')

true = pd.read_csv('True.csv')

In [7]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
fake['class']=0
true['class']=1

In [10]:
data = pd.concat([fake,true], axis = 0)

In [11]:
data.sample(10)

,title,text,subject,date,class
4079,Trump opposes including Obamacare subsidies in...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"April 26, 2017",1
12027,Belgian trial of Paris attack suspect postpone...,BRUSSELS (Reuters) - The Belgian trial of a ke...,worldnews,"December 18, 2017",1
11975,Eurosceptics baulk as May pitches status quo B...,LONDON (Reuters) - British Prime Minister Ther...,worldnews,"December 18, 2017",1
3660,George Takei Just B*tch-Slapped Trump After H...,"If you ve been paying attention, you know that...",News,"November 27, 2016",0
9027,"Seeking to regain ground, Trump calls Clinton ...",NEW YORK (Reuters) - Republican presidential h...,politicsNews,"June 22, 2016",1
1047,Two Republican senators blast Trump as party f...,WASHINGTON (Reuters) - Tensions among Republic...,politicsNews,"October 24, 2017",1
5118,Baghdad relieved by Trump's exclusion of Iraq ...,BAGHDAD (Reuters) - Iraq expressed “deep relie...,politicsNews,"March 6, 2017",1
11458,"Myanmar, accused of crackdown, invited to U.S....","WASHINGTON (Reuters) - The Myanmar military, w...",worldnews,"December 23, 2017",1
4152,World Bank's Kim says Trump likes lender's pri...,WASHINGTON (Reuters) - In an era of shrinking ...,politicsNews,"April 20, 2017",1
10026,WATCH: NBC’S CHUCK TODD Attempts To “Normalize...,Here is an excerpt from their coverage of Todd...,politics,"Aug 28, 2017",0


In [12]:
data =data.drop(["title","subject","date"], axis = 1)

In [16]:
data.reset_index(inplace=True)

In [17]:
data.drop(['index'],axis = 1, inplace=True)

In [18]:
data.sample(5)

,level_0,text,class
29482,29482,PHILADELPHIA (Reuters) - U.S. President Donald...,1
10383,10383,Low blow! Political hack Angela Rye refused to...,0
21913,21913,THE CONGRESS IS THERE TO DO WHAT S BEST FOR TH...,0
35024,35024,JOHANNESBURG (Reuters) - South Africa s Presid...,1
21118,21118,"The citizens of Rheinberg, Germany can thank t...",0


In [20]:
def clean_text(text):
  text = text.lower()
  text = re.sub('\[.*?\]',"", text)
  text = re.sub("\\W"," ",text)
  text = re.sub("https?:://\S+|www\.\S+","",text)
  text = re.sub("<.*?>+","", text)
  text = re.sub("[%s]" % re.escape(string.punctuation),"", text)
  text = re.sub("\n", "", text)
  text = re.sub("\w*\d\w*","", text)
  return text

In [21]:
data["text"] = data["text"].apply(clean_text)

In [22]:
x=data["text"]
y=data["class"]

xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.25, random_state=42)

In [23]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [24]:
lr = LogisticRegression()
lr.fit(xv_train, ytrain)

LogisticRegression()

In [25]:
prediction = lr.predict(xv_test)
lr.score(xv_test, ytest)

0.9858351893095768

In [26]:
print(classification_report(ytest, prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [27]:
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(lr,"lr_model.jb")

['lr_model.jb']